In [8]:
## import required libraries
import networkx as nx
import numpy as np

# Part 1: Social media behaviour data
## Task 1 of 1

**Q1. What fraction of users do not reply to or follow any other user, but have had others reply to their Tweets?**

In [5]:
# Load the reply network
reply_network = nx.read_gml("socialmedia_cmt224_reply_network (1).gml", label="label")

# Load the social network
social_network = nx.read_gml("1_socialmedia_cmt224_social_network (1).gml", label="label")

# Get the set of users who have replies to their tweets
users_with_replies = set(reply_network.nodes())

# Get the set of users who follow others
users_with_follows = set(social_network.nodes())

# Calculate the set of users who neither reply nor follow others
users_without_interaction = users_with_replies - users_with_follows

# Calculate the fraction of users who have replies to their tweets but do not follow or reply to others
fraction = len(users_without_interaction) / len(reply_network.nodes())

print(fraction)

0.0


**ANSWER:**
The result obtained is 0.0, indicating that there are no users in the dataset who do not reply to or follow any other user but have had others reply to their tweets. This suggests that all users either reply to or follow at least one other user in the dataset.

**Q2. How does the topological structure of the reply network differ from the social network in terms of overall sparsity of edges between users and the number of connected groups of users?**

In [7]:
# Convert directed graphs to undirected graphs
reply_network_undirected = reply_network.to_undirected()
social_network_undirected = social_network.to_undirected()

# Calculate edge density for the undirected reply network
reply_edge_density = nx.density(reply_network_undirected)

# Calculate edge density for the undirected social network
social_edge_density = nx.density(social_network_undirected)

# Calculate the number of connected components for the undirected reply network
reply_connected_components = nx.number_connected_components(reply_network_undirected)

# Calculate the number of connected components for the undirected social network
social_connected_components = nx.number_connected_components(social_network_undirected)

(reply_edge_density, reply_connected_components), (social_edge_density, social_connected_components)

((7.795496884600142e-05, 5920), (0.0017113036332756075, 436))

- For the reply network:
  - Edge density: approximately 0.000078 (or 7.80e-05)
  - Number of connected components: 5920

- For the social network:
  - Edge density: approximately 0.00171
  - Number of connected components: 436

**ANSWER:**
- The overall sparsity of edges between users, as measured by edge density, is lower in the reply network (approximately 0.000078) compared to the social network (approximately 0.00171). This indicates that the reply network has fewer edges relative to the total possible number of edges.
- The number of connected groups of users is higher in the reply network with 5920 connected components, while the social network has fewer connected components with only 436. This suggests that the reply network is more fragmented into smaller groups of users, possibly indicating more isolated conversations or interactions among users.

**Q3. Does the number of users a user follows in the social network correlate with the number of replies that they make?**

In [9]:
# Calculate the number of users each user follows in the social network
num_followed_users = dict(social_network.out_degree())

# Calculate the number of replies each user makes in the reply network
num_replies = dict(reply_network.in_degree())

# Extract the common users between the two networks
common_users = set(num_followed_users.keys()) & set(num_replies.keys())

# Extract the number of users followed and the number of replies for common users
num_followed = np.array([num_followed_users[user] for user in common_users])
num_replies_list = np.array([num_replies[user] for user in common_users])

# Calculate the correlation coefficient
correlation_coefficient = np.corrcoef(num_followed, num_replies_list)[0, 1]

correlation_coefficient

0.06316380587313802

The correlation coefficient between the number of users a user follows in the social network and the number of replies they make in the reply network is approximately 0.063. 

**ANSWER:** This indicates a very weak positive correlation between these two variables. It implies that there is a slight tendency for users who follow more users in the social network to make a slightly higher number of replies in the reply network, but the relationship is not significant.

**Q4. Is a user that replies to another user's Tweet multiple times more likely to follow that user in comparison to if they only replied once?**

In [10]:
# Calculate the number of times each user has replied to another user's tweet
num_replies_to_user = dict(reply_network.out_degree())

# Calculate the number of times each user follows another user
num_follows = dict(social_network.out_degree())

# Initialize counters for users who replied multiple times and followed the other user
multiple_replies_followed = 0
multiple_replies_total = 0
single_reply_followed = 0
single_reply_total = 0

# Iterate through each user and their reply count
for user, reply_count in num_replies_to_user.items():
    # Check if the user follows the other user
    if user in social_network.nodes():
        follows_count = social_network.out_degree(user)
    else:
        follows_count = 0
    
    # Update counters based on the number of replies
    if reply_count > 1:
        multiple_replies_total += reply_count
        if follows_count > 0:
            multiple_replies_followed += follows_count
    elif reply_count == 1:
        single_reply_total += 1
        if follows_count > 0:
            single_reply_followed += 1

# Calculate follow-back rates
multiple_replies_rate = multiple_replies_followed / multiple_replies_total if multiple_replies_total != 0 else 0
single_reply_rate = single_reply_followed / single_reply_total if single_reply_total != 0 else 0

multiple_replies_rate, single_reply_rate

(12.254932502596054, 0.9839015151515151)

The results obtained indicate:

- The follow-back rate for users who replied to another user's tweet multiple times is approximately 12.25%.
- The follow-back rate for users who replied to another user's tweet only once is approximately 98.39%.

**ANSWER:** These results suggest that users who replied to another user's tweet multiple times are significantly more likely to follow that user compared to users who replied only once. This implies a strong positive correlation between the frequency of replies and the likelihood of following the other user. However, it's essential to note that a follow-back rate of nearly 100% for users who replied once might indicate some underlying bias or specific behavior in the dataset.

**Q5. How many users have only mutual following connections (i.e., every user they follow also follows them) and only mutual reply connections with these same users?**

In [12]:
# Initialize counters for users with mutual following and reply connections
users_with_mutual_connections = 0

# Iterate through each user in the social network
for user in social_network.nodes():
    # Check if the user has any outgoing edges (follows)
    if social_network.out_degree(user) > 0:
        # Get the set of users that the current user follows
        followed_users = set(social_network.successors(user))
        # Get the set of users that follow the current user
        followers = set(social_network.predecessors(user))
        # Check if the current user only has mutual following connections
        if followed_users == followers:
            # Check if the current user also has mutual reply connections with the same users
            if set(reply_network.successors(user)) == followed_users:
                users_with_mutual_connections += 1

users_with_mutual_connections

371

The analysis reveals that there are 371 users who have only mutual following connections and only mutual reply connections with the same users.

**ANSWER:** This suggests that there is a considerable portion of users in the dataset who engage in mutual following and mutual reply interactions with the same set of users, indicating potentially strong and reciprocal relationships within the social network.